In [ ]:
# Install necessary packages
!pip install -q gradio transformers

In [ ]:
# Import required libraries
import torch
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Load token and model
model_name = "google/gemma-2-2b-it"
hf_token = "use your Huggingface token here"  # Replace with your Huggingface token

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_token).to(device)

In [ ]:
# Format and manage conversation history
def format_conversation(history):
    conversation = ""
    for user_msg, bot_msg in history:
        conversation += f"User: {user_msg}\nBot: {bot_msg}\n"
    return conversation

In [ ]:
def chatbot_response(user_input, history):
    # Build the conversation prompt
    conversation = format_conversation(history)
    prompt = conversation + f"User: {user_input}\nBot:"

    # Tokenize and generate response
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    bot_reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean the response: remove prompt from output
    if "Bot:" in bot_reply:
        bot_reply = bot_reply.split("Bot:")[-1].strip()

    return bot_reply

In [ ]:
def chatbot_interface(user_input, history):
    response = chatbot_response(user_input, history)  # Make sure history is passed correctly
    history.append((user_input, response))
    return history, ""

# Creating the Gradio Interface
demo = gr.Blocks()

with demo:
    gr.Markdown("## GenAI Chatbot")
    chatbot = gr.Chatbot()
    with gr.Row():
        user_input = gr.Textbox(placeholder="Type your message here...", lines=1)
        send_button = gr.Button("Send")

    history = gr.State([])

    send_button.click(chatbot_interface, inputs=[user_input, history], outputs=[chatbot, user_input])
    user_input.submit(chatbot_interface, inputs=[user_input, history], outputs=[chatbot, user_input])


In [ ]:
# Launch the chatbot
demo.launch(share = True, debug = True)